必要な機能

元画像の読み込みと処理
マッピングされた画像の読み込み
バウンティボックスの情報の読み込み
処理後のバウンティボックスの座標の算出
指標の算出（2つ）

In [13]:
from torchvision import transforms
from PIL import Image

class Experience_bounding_box():
    
    def __init__(self) -> None:
        """
        初期化
        """
        self.transform = transforms.Compose(
            [
                transforms.Resize(256),  # (256, 256) で切り抜く。
                transforms.CenterCrop(224),  # 画像の中心に合わせて、(224, 224) で切り抜く
            ]
        )

 
    def open_and_resize_original_image(self, img_path : str) -> Image.Image:
        """
        img_pathの元画像を読み込んでサイズを(244, 244)にして返す
        """
        original_image = Image.open(img_path)
        original_image = self.transform(original_image)
        
        return original_image
    
    
    def open_result_image(self, img_path : str) -> Image.Image:
        """
        img_pathのマッピングされた画像を読み込んで返す（サイズの変更はしない）
        """
        result_image = Image.open(img_path)
        result_image = self.transform(result_image)
        
        return result_image
    
    
            

In [14]:
exp1 = Experience_bounding_box()

In [15]:
img = exp1.open_original_image("cat.jpg")

In [16]:
type(img)

PIL.Image.Image

In [12]:
img.getpixel((0,0))

(188, 174, 171)